In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import font_manager
import sklearn
import pprint
import re
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')

font_fname = 'C:/Windows/Fonts/malgun.ttf'
font_family = font_manager.FontProperties(fname=font_fname).get_name()

plt.rcParams["font.family"] = font_family
plt.rcParams["axes.unicode_minus"] = False


%matplotlib inline

In [65]:
price_df = pd.read_excel('부동산데이터_수도권_202308.xlsx')

In [66]:
check_col = ['매매하위평균가', '매매상위평균가', '매매/전세가', '매매호가', '전세호가']

price_df.dropna(subset=check_col ,how='all', inplace=True)

In [4]:
price_df.shape

(52186, 58)

용인처인구 -> 용인시

In [67]:
city_cols = ['화성시','오산시','용인시','평택시','수원시','안성시', '용인처인구']
filtered_df = price_df[price_df['법정동주소'].str.contains('|'.join(city_cols)) | price_df['도로명주소'].str.contains('|'.join(city_cols))]

In [68]:
filtered_df.shape

(7078, 58)

In [69]:
filtered_df[['법정동주소','도로명주소']]

,법정동주소,도로명주소
71119,경기도 수원시 권선구 고색동 306-1,경기도 수원시 권선구 고현로5번길 25
71120,경기도 수원시 권선구 고색동 306-1,경기도 수원시 권선구 고현로5번길 25
71121,경기도 수원시 권선구 고색동 306-1,경기도 수원시 권선구 고현로5번길 25
71122,경기도 수원시 권선구 고색동 886-83,경기도 수원시 권선구 매송고색로673번길 11
71123,경기도 수원시 권선구 고색동 886-83,경기도 수원시 권선구 매송고색로673번길 11
...,...,...
94110,경기도 화성시 향남읍 하길리 1464,경기도 화성시 상신하길로328번길 21
94111,경기도 화성시 향남읍 하길리 1464,경기도 화성시 상신하길로328번길 21
94112,경기도 화성시 향남읍 하길리 1464,경기도 화성시 상신하길로328번길 21
94113,경기도 화성시 향남읍 하길리 1464,경기도 화성시 상신하길로328번길 21


In [70]:
house_df = pd.read_csv('apartment_rail_v7.csv')

In [71]:
house_ols_df = house_df.copy()
house_ols_df['입주예정일'] = house_df['입주예정일'].astype(str).str[:4]
house_ols_df['대형건설사'] = house_ols_df['대형건설사'].apply(lambda x: True if x == 'YES' else False)
house_ols_df['법정동주소'] = house_ols_df['법정동주소'].astype(str)

In [72]:
filtered_house_df = house_ols_df[house_ols_df['법정동주소'].str.contains('|'.join(city_cols))]

In [32]:
filtered_house_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 805 entries, 37 to 3020
Data columns (total 42 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   아파트명         805 non-null    object 
 1   법정동주소        805 non-null    object 
 2   위도           805 non-null    float64
 3   경도           805 non-null    float64
 4   세대수          805 non-null    int64  
 5   임대세대수        742 non-null    float64
 6   최고층          801 non-null    float64
 7   최저층          792 non-null    float64
 8   최대공급면적       805 non-null    float64
 9   최소공급면적       805 non-null    float64
 10  총아파트동수       801 non-null    float64
 11  용적률          457 non-null    float64
 12  건폐율          415 non-null    float64
 13  세대평균_주차대수    793 non-null    float64
 14  공급면적         805 non-null    float64
 15  전용면적         805 non-null    float64
 16  전용율          805 non-null    float64
 17  방수           793 non-null    float64
 18  욕실수          793 non-null    float64
 19  현관구조  

In [75]:
merged_df = filtered_df[filtered_df['법정동주소'].isin(filtered_house_df['법정동주소']) |
                       filtered_df['도로명주소'].isin(filtered_house_df['법정동주소'])]

In [76]:
merged_df.shape

(105, 58)

In [77]:
merged_df2 = pd.merge(filtered_df, filtered_house_df, on=['위도', '경도'], how='inner')

In [78]:
filtered_df['lat_lon'] = list(zip(filtered_df['위도'], filtered_df['경도']))
filtered_house_df['lat_lon'] = list(zip(filtered_house_df['위도'], filtered_house_df['경도']))

# filtered_df와 filtered_house_df의 'lat_lon' 컬럼을 기준으로 인덱스 매칭
index_matches = filtered_df.index[filtered_df['lat_lon'].isin(filtered_house_df['lat_lon'])]
filtered_df.drop('lat_lon', axis=1, inplace=True)
filtered_house_df.drop('lat_lon', axis=1, inplace=True)

In [79]:
index_matches

Int64Index([72208, 72210, 72212, 72213, 72624, 72625, 72626, 72627, 72629,
            72630, 72632, 72633, 72634, 73034, 73035, 73036, 73037, 73038,
            73039, 73040, 73459, 73462, 73465, 73467, 73889, 73890, 73892,
            73895, 73897, 74239, 74240, 74241, 74242, 74243, 74244, 74245,
            74246, 74247, 83069, 85025, 85026, 85027, 85028, 85060, 85061,
            85062, 85064, 85067, 85068, 85069, 85070, 85071, 85072, 89222,
            89233, 89234, 89235, 89236, 89237, 89238, 89276, 89277, 89278,
            89279, 90154, 91222, 91223, 91224, 92054, 92114, 92115, 92116,
            92758, 92759, 92760, 92761, 92788, 92789, 92790, 93592],
           dtype='int64')

In [86]:
merged_df3 = filtered_df.loc[index_matches]

In [83]:
merged_df.shape

(105, 58)

In [87]:
merged_df3.shape

(80, 58)

In [88]:
combined_df = pd.concat([merged_df, merged_df3]).drop_duplicates()

In [89]:
combined_df.shape

(105, 58)

In [90]:
price_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52186 entries, 4 to 106016
Data columns (total 58 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   타입         52186 non-null  object 
 1   아파트명       52186 non-null  object 
 2   면적         52186 non-null  object 
 3   법정동주소      52186 non-null  object 
 4   도로명주소      52186 non-null  object 
 5   위도         52186 non-null  float64
 6   경도         52186 non-null  float64
 7   세대수        52186 non-null  int64  
 8   임대세대수      52186 non-null  int64  
 9   영구임대세대수    52186 non-null  int64  
 10  국민임대세대수    52186 non-null  int64  
 11  민간임대세대수    52186 non-null  int64  
 12  공공임대세대수    52186 non-null  int64  
 13  장기임대세대수    52186 non-null  int64  
 14  기타임대세대수    52186 non-null  int64  
 15  최고층        52186 non-null  int64  
 16  최저층        52186 non-null  int64  
 17  사용승인일      52178 non-null  float64
 18  최대공급면적     52186 non-null  float64
 19  최소공급면적     52186 non-null  float64
 20  총아파트동

In [106]:
# 컬럼 값 변환 함수 정의
def convert_to_integer_range(value):
    if pd.notna(value):  # NaN이 아닌 경우에만 변환 적용
        if '~' in value:
            min_value, max_value = value.split('~')
            min_value = int(min_value.replace('억', '').replace(',', ''))
            max_value = int(max_value.replace('억', '').replace(',', ''))
            return f"{min_value:,} ~ {max_value:,}"
        else:
            return f"{int(value.replace('억', '').replace(',', '')):,}"
    else:
        return value  # NaN 값은 그대로 반환

# 매매호가 컬럼의 모든 값을 변환 적용
price_df['매매호가_분리'] = price_df['매매호가'].apply(convert_to_integer_range)

In [107]:
price_df['매매호가_분리']

4              17 ~ 18
5         188,000 ~ 21
6              20 ~ 22
7              22 ~ 25
8         235,000 ~ 28
              ...     
106012          24,000
106013          26,000
106014      3 ~ 35,000
106015             NaN
106016             NaN
Name: 매매호가_분리, Length: 52186, dtype: object

In [99]:
list(price_df['매매호가'].unique())

['17억~18억',
 '18억 8,000~21억',
 '20억~22억',
 '22억~25억',
 '23억 5,000~28억',
 '25억~28억 5,000',
 nan,
 '26억~27억',
 '29억 5,000~37억',
 '30억~35억',
 '33억~39억',
 '39억',
 '40억~41억',
 '41억',
 '41억~41억 5,000',
 '31억~37억',
 '69억',
 '25억~30억',
 '30억~32억',
 '32억~35억',
 '35억~36억',
 '12억 4,000~13억 7,000',
 '16억',
 '23억~24억',
 '30억',
 '22억',
 '21억 9,000',
 '39억~42억',
 '39억 5,000~43억',
 '22억 5,000~24억',
 '25억 8,000~27억',
 '28억~29억',
 '19억~20억',
 '20억~21억',
 '23억 5,000~25억',
 '26억 5,000~27억',
 '19억 4,000~19억 5,000',
 '20억 5,000~22억',
 '24억~25억',
 '27억',
 '18억 5,000~19억',
 '25억',
 '23억 5,000~24억',
 '23억 5,000',
 '23억 5,000~26억',
 '31억',
 '33억',
 '36억 5,000~40억',
 '36억 5,000~38억',
 '11억~13억',
 '13억 5,000~15억',
 '16억~17억',
 '9억 8,000~10억 5,000',
 '11억 8,000~13억',
 '13억 7,000~14억 7,000',
 '23억 7,000~28억',
 '24억~28억',
 '24억~27억',
 '17억',
 '17억~17억 5,000',
 '20억 5,000~24억 5,000',
 '21억~22억',
 '20억 7,000~23억',
 '26억 5,000~29억',
 '29억 5,000',
 '28억 4,000~32억',
 '28억~30억 5,000',
 '28억~30억',
 '28억 5,000~30억',
 '36억 5

In [113]:
def process_value(value):
    if pd.isna(value):
        return value
    
    parts = value.split('~')
    if len(parts) == 1:
        print(parts)
        return parts[0].strip()
    else:
        sub_parts = [part.strip() for part in parts]
        if len(sub_parts) == 2:
            return sub_parts
        else:
            return value


price_df['매매호가'] = price_df['매매호가'].apply(process_value)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [109]:
price_df['매매호가']

4               [17억, 18억]
5         [18억 8,000, 21억]
6               [20억, 22억]
7               [22억, 25억]
8         [23억 5,000, 28억]
                ...       
106012            2억 4,000
106013            2억 6,000
106014      [3억, 3억 5,000]
106015                 NaN
106016                 NaN
Name: 매매호가, Length: 52186, dtype: object